In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_MITOSISi/Expression_CNA/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_MITOSISi/Expression_CNA/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (MITOSISi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (MITOSISi)

GDSCE = pd.read_csv("GDSC_exprs.z.MITOSISi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC CNA dataset (MITOSISi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [9]:
# read GDSC CNA dataset (MITOSISi)

GDSCC = pd.read_csv("GDSC_CNA.MITOSISi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

GDSCC.head()

gene_id,1,2,9,10,12,13,14,15,16,18,...,107985535,107985759,107986588,107986809,107986898,107987337,107987341,109731405,112441434,121676927
683665,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.807355,0.000000,0.000000,0.000000
684055,0.000000,0.584963,0.584963,0.584963,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.584963,0.000000,0.584963,0.584963,-4.32,-4.320000,0.000000,0.000000,0.584963
684057,-0.415037,0.000000,0.584963,0.584963,-1.000000,0.000000,0.0,0.0,-0.415037,-0.415037,...,-0.415037,0.000000,0.000000,0.000000,0.807355,-1.00,-1.000000,-0.415037,-0.415037,0.807355
684059,0.000000,0.000000,0.000000,0.000000,0.000000,0.415037,0.0,0.0,0.000000,0.415037,...,0.000000,0.000000,0.415037,0.000000,0.415037,-4.32,-4.320000,0.000000,0.000000,0.736966
684062,-0.415037,0.000000,0.321928,0.321928,-0.415037,-0.415037,0.0,0.0,-0.415037,0.000000,...,-0.415037,0.000000,0.000000,0.000000,0.321928,-4.32,-4.320000,-0.415037,-1.000000,0.321928


In [10]:
# change directory to read GDSC response dataset (MITOSISi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# read GDSC response dataset (MITOSISi)

GDSCR = pd.read_csv("GDSC_response.MITOSISi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,0.70402689461727,VX-680,1,1,1
683665,0,-3.64728727366553,Paclitaxel,1,1,1
683665,0,0.307772732941787,S-Trityl-L-cysteine,1,1,1
683665,0,-1.86219201025588,BI-2536,1,1,1
683665,0,-3.03887798334128,GW843682X,1,1,1


In [12]:
# print count of cell lines for each drug (MITOSISi)

GDSCR["drug"].value_counts()

drug
Genentech Cpd 10       771
SB-715992              770
NPK76-II-72-1          769
MPS-1-IN-1             767
GSK1070916             747
ZM-447439              694
Paclitaxel             370
GW843682X              370
S-Trityl-L-cysteine    369
BI-2536                368
VX-680                 365
Name: count, dtype: int64

In [13]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Paclitaxel)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Paclitaxel)

TCGAEpaclitaxel = pd.read_csv("TCGA_exprs.Paclitaxel.eb_with.GDSC_exprs.Paclitaxel.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEpaclitaxel = pd.DataFrame.transpose(TCGAEpaclitaxel)
TCGAEpaclitaxel.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-50-5068-01A,6.07077606939576,4.22435549873448,6.09703453175451,2.79409553041956,2.16511492655888,6.23897149546054,7.50971824388108,2.62136482478483,6.78739521681224,2.98021984531206,...,9.80444787373941,2.79886640790268,2.81581286530484,3.00085815257311,2.80511713535762,2.40148864399763,2.58029798170468,3.80309726320581,2.93775838077045,3.15136733303223
TCGA-53-7624-01A,3.36417856158271,1.84257380220382,4.89461611130517,3.01203072800212,1.08675483906125,2.65397613649159,8.19887134647799,2.64492131772784,9.57693655417731,2.75400746416671,...,9.87655475612095,2.79886640790268,2.81581286530484,3.47072364084828,2.92631228926461,3.04342978048926,3.85945702517433,2.76860237341666,4.70753879715127,4.40422733195751
TCGA-99-8033-01A,3.17759663469389,4.069358321802,5.30291306958158,3.04306111581331,1.71759129803103,2.9492322528108,8.35388516673678,2.61215713067395,9.55148283225251,2.62363654845594,...,9.97545462791267,2.79886640790268,2.81581286530484,3.61434933029633,2.82800183535407,2.62335146413649,2.82601101115657,2.76860237341666,4.83367880098437,4.96091320806586


In [15]:
# change directory to read TCGA CNA dataset (Paclitaxel)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [16]:
# read TCGA CNA dataset (Paclitaxel)

TCGACpaclitaxel = pd.read_csv("TCGA_CNA.Paclitaxel.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGACpaclitaxel.drop_duplicates(keep='last')
TCGACpaclitaxel = pd.DataFrame.transpose(TCGACpaclitaxel)
TCGACpaclitaxel = TCGACpaclitaxel.loc[:, ~TCGACpaclitaxel.columns.duplicated()]

TCGACpaclitaxel.head(3)

gene,1,2,3,9,10,11,12,13,14,15,...,100653016,100653017,100653018,100653019,100653020,100653021,100653022,100653023,100653024,100653025
TCGA-50-5068-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-53-7624-01A,1,0,0,-1,-1,-1,0,0,0,1,...,-1,1,1,-1,1,0,1,-1,-1,1
TCGA-99-8033-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# variance threshold for GDSC expression dataset (MITOSISi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC CNA dataset (MITOSISi)

GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [19]:
# select shared genes between GDSC and TCGA expression and CNA datasets 

ls = GDSCE.columns.intersection(GDSCC.columns)
ls = ls.intersection(TCGAEpaclitaxel.columns)
ls = ls.intersection(TCGACpaclitaxel.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA # of common genes: {ls.shape[0]}")

GDSC and TCGA # of common genes: 17869


In [20]:
# select shared samples between GDSC expression and response datasets (MITOSISi)

ls2 = GDSCE.index.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 799


In [21]:
# subset shared genes and samples in GDSC expression, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (799, 17869)
GDSC # of common samples and genes (CNA): (799, 17869)
GDSC # of samples (Response): 6360


In [22]:
# select shared samples between TCGA expression and CNA datasets (Paclitaxel) 

ls3 = TCGAEpaclitaxel.index.intersection(TCGACpaclitaxel.index)

print(f"TCGA # of common samples (Paclitaxel): {ls3.shape[0]}")

TCGA # of common samples (Paclitaxel): 35


In [23]:
# select shared genes and samples between TCGA expression and CNA datasets (Paclitaxel)

TCGAEpaclitaxel = TCGAEpaclitaxel.loc[ls3, ls]
TCGACpaclitaxel = TCGACpaclitaxel.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Paclitaxel): {TCGAEpaclitaxel.shape}")
print(f"TCGA # of common samples and genes for CNA (Paclitaxel): {TCGACpaclitaxel.shape}\n")

TCGA # of common samples and genes for Expression (Paclitaxel): (35, 17869)
TCGA # of common samples and genes for CNA (Paclitaxel): (35, 17869)



In [24]:
# change directory to read DEGs (MITOSISi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi


In [25]:
# read DEGs (MITOSISi)

DEGs_filtered_data = pd.read_excel("MITOSISi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among mitotic inhibitors\n")

DEGs_filtered_data.head()

There are 996 DEGs (p-adjusted < 0.05 and |logFC| > 1) among mitotic inhibitors



,Gene Symbol,Frequency
0,PLS3,8
1,GMFG,7
2,GPRC5A,7
3,TJP1,7
4,CXCR4,7


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id = pd.Series(DEGs_entrez_id)

result = mg.query("SLC22A18", species="human", scopes="symbol", fields="entrezgene")
DEGs_entrez_id = pd.concat([DEGs_entrez_id, pd.Series(str(result["hits"][1]["entrezgene"]))], ignore_index=True)

DEGs_entrez_id.shape[0]

2 input query terms found dup hits:	[('CAST', 2), ('CLECL1P', 2)]
1 input query terms found no hit:	['SLC22A18']


996

In [27]:
# assign GDSC data to new variables

exprs_z = GDSCE
cna = GDSCC
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,0.70402689461727,VX-680,1,1,1
683665,0,-3.64728727366553,Paclitaxel,1,1,1
683665,0,0.307772732941787,S-Trityl-L-cysteine,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'BI-2536',
 'GSK1070916',
 'GW843682X',
 'Genentech Cpd 10',
 'MPS-1-IN-1',
 'NPK76-II-72-1',
 'Paclitaxel',
 'S-Trityl-L-cysteine',
 'SB-715992',
 'VX-680',
 'ZM-447439'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
        949167,  998184, 1290771, 1298156, 1330948, 1330993, 1503364, 1509073,
       1659818, 1659819],
      dtype='int64', length=6360)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
leukemia             696
lymphoma             626
lung_NSCLC           600
urogenital_system    538
lung_SCLC            433
aero_dig_tract       424
nervous_system       416
skin                 371
breast               311
bone                 302
large_intestine      295
digestive_system     282
neuroblastoma        276
kidney               158
pancreas             156
myeloma              145
soft_tissue          141
lung                 103
thyroid               87
Name: count, dtype: int64

In [32]:
# list count of cell lines as to tissues treated with paclitaxel

filter = (dat["drug"] == "Paclitaxel")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

GDSC\nTissue descriptor 1
leukemia             64
lymphoma             57
lung_SCLC            35
nervous_system       25
neuroblastoma        23
lung_NSCLC           20
bone                 20
skin                 19
urogenital_system    16
large_intestine      15
aero_dig_tract       14
myeloma              13
digestive_system     12
breast               11
kidney               10
soft_tissue           9
lung                  3
pancreas              2
thyroid               2
Name: count, dtype: int64

In [33]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '949167', '998184', '1290771', '1298156', '1330948', '1330993',
       '1503364', '1509073', '1659818', '1659819'],
      dtype='object', length=6360)

In [34]:
# filter GDSC expression and CNA datasets (MITOSISi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    expression_zscores.append(e_z)
    CNA.append(c)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCCv2 = GDSCCv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEpaclitaxel = TCGAEpaclitaxel.loc[:,ls4]
TCGACpaclitaxel = TCGACpaclitaxel.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for Paclitaxel (Expression): {TCGAEpaclitaxel.shape}")
print(f"TCGA # of common samples and genes for Paclitaxel (CNA): {TCGACpaclitaxel.shape}\n")

GDSC # of common samples and genes (Expression): (6360, 984)
GDSC # of common samples and genes (CNA): (6360, 984)
GDSC # of common samples (Response): 6360

TCGA # of common samples and genes for Paclitaxel (Expression): (35, 984)
TCGA # of common samples and genes for Paclitaxel (CNA): (35, 984)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 0 1]


In [36]:
# change directory to read TCGA response dataset (Paclitaxel)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read TCGA response dataset (Paclitaxel)

TCGARpaclitaxel = pd.read_csv("TCGA_response.Paclitaxel.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARpaclitaxel.dropna(inplace=True)
TCGARpaclitaxel.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARpaclitaxel["response"] = TCGARpaclitaxel.loc[:, "response"].apply(lambda x: d[x])
Ytspaclitaxel = TCGARpaclitaxel["response"].values    

print(f"There are {Ytspaclitaxel.shape[0]} samples with response data in the TCGA (Paclitaxel)\n")
Ytspaclitaxel

There are 35 samples with response data in the TCGA (Paclitaxel)



array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [38]:
# TCGA cohort table with TCGA response dataset (Paclitaxel)

dat_tcga = TCGARpaclitaxel["cohort"]

dat_tcga.value_counts()

cohort
BRCA    27
LUAD     3
UCS      3
STAD     1
HNSC     1
Name: count, dtype: int64

In [39]:
# filter BRCA cohort samples (Paclitaxel)

filter = (TCGARpaclitaxel["cohort"] == "BRCA")
tcga_brca_samples = dat_tcga.loc[filter,].index
tcga_brca_samples

Index(['TCGA-A2-A0EP-01A', 'TCGA-A2-A3XS-01A', 'TCGA-A2-A3XW-01A',
       'TCGA-A2-A3XX-01A', 'TCGA-A2-A3XY-01A', 'TCGA-A2-A4S2-01A',
       'TCGA-A7-A4SA-01A', 'TCGA-A7-A4SE-01A', 'TCGA-A7-A5ZV-01A',
       'TCGA-A7-A5ZX-01A', 'TCGA-A8-A08X-01A', 'TCGA-A8-A09M-01A',
       'TCGA-AQ-A54O-01A', 'TCGA-B6-A402-01A', 'TCGA-D8-A3Z6-01A',
       'TCGA-EW-A1OY-01A', 'TCGA-EW-A1P3-01A', 'TCGA-EW-A2FR-01A',
       'TCGA-GM-A2DA-01A', 'TCGA-GM-A2DB-01A', 'TCGA-GM-A2DF-01A',
       'TCGA-GM-A2DH-01A', 'TCGA-GM-A2DM-01A', 'TCGA-GM-A2DN-01A',
       'TCGA-GM-A3XG-01A', 'TCGA-GM-A3XL-01A', 'TCGA-GM-A3XN-01A'],
      dtype='object', name='sample')

In [40]:
# filter responses of BRCA cohort samples (Paclitaxel)

Ytspaclitaxel_brca = TCGARpaclitaxel.loc[tcga_brca_samples,"response"].values
Ytspaclitaxel_brca

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1])

In [41]:
# count of sensitive and resistance BRCA samples (Paclitaxel)

TCGARpaclitaxel.loc[tcga_brca_samples,"response"].value_counts()

response
1    22
0     5
Name: count, dtype: int64

In [42]:
# define maximum iteration

max_iter = 50

In [43]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 64
hdm2 = 32
rate1 = 0.5
rate2 = 0.6
rate3 = 0.6

In [44]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEpaclitaxel = scalerGDSC.transform(TCGAEpaclitaxel.values)    

X_testEpaclitaxel = scalerGDSC.transform(TCGAEpaclitaxel.values)    
X_testEpaclitaxel_brca = scalerGDSC.transform(TCGAEpaclitaxel.loc[tcga_brca_samples,].values)    

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCpaclitaxel = np.nan_to_num(TCGACpaclitaxel.values.astype(np.float32))
X_testCpaclitaxel_brca = np.nan_to_num(TCGACpaclitaxel.loc[tcga_brca_samples,].values)

TX_testEpaclitaxel = torch.FloatTensor(X_testEpaclitaxel)
TX_testCpaclitaxel = torch.FloatTensor(X_testCpaclitaxel.astype(np.float32))
ty_testEpaclitaxelorouracil = torch.FloatTensor(Ytspaclitaxel.astype(int))

TX_testEpaclitaxel = torch.FloatTensor(X_testEpaclitaxel)
TX_testEpaclitaxel_brca = torch.FloatTensor(X_testEpaclitaxel_brca)

TX_testCpaclitaxel = torch.FloatTensor(X_testCpaclitaxel.astype(np.float32))
TX_testCpaclitaxel_brca = torch.FloatTensor(X_testCpaclitaxel_brca.astype(np.float32))
n_sampE, IE_dim = X_trainE.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA.pt")
AutoencoderC = torch.load(models_dir + "CNA_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA.pt")

Clas = torch.load(models_dir + "Class_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA.pt")

AutoencoderE.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_MITOSISi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_MITOSISi): {round(AUCTaucpr,2)}\n")

ZETpaclitaxel = AutoencoderE(TX_testEpaclitaxel)
ZCTpaclitaxel = AutoencoderC(TX_testCpaclitaxel)
ZTTpaclitaxel = torch.cat((ZETpaclitaxel, ZCTpaclitaxel), 1)
ZTTpaclitaxel = F.normalize(ZTTpaclitaxel, p=2, dim=0)
PredTpaclitaxel = Clas(ZTTpaclitaxel)
AUCTpaclitaxel = roc_auc_score(Ytspaclitaxel.astype(int), PredTpaclitaxel.detach().numpy())
#print(f"TCGA - Test AUC (Paclitaxel): {round(AUCTpaclitaxel,2)}")
AUCTpaclitaxel_aucpr = average_precision_score(Ytspaclitaxel.astype(int), PredTpaclitaxel.detach().numpy())
#print(f"TCGA - Test AUCPR (Paclitaxel): {round(AUCTpaclitaxel_aucpr,2)}\n")

ZETpaclitaxel_brca = AutoencoderE(TX_testEpaclitaxel_brca)
ZCTpaclitaxel_brca = AutoencoderC(TX_testCpaclitaxel_brca)
ZTTpaclitaxel_brca = torch.cat((ZETpaclitaxel_brca,  ZCTpaclitaxel_brca), 1)
ZTTpaclitaxel_brca = F.normalize(ZTTpaclitaxel_brca, p=2, dim=0)
PredTpaclitaxel_brca = Clas(ZTTpaclitaxel_brca)
AUCTpaclitaxel_brca = roc_auc_score(Ytspaclitaxel_brca.astype(int), PredTpaclitaxel_brca.detach().numpy())
#print(f"TCGA - Test AUC (Paclitaxel, BRCA): {round(AUCTpaclitaxel_brca,2)}")
AUCTpaclitaxel_brca_aucpr = average_precision_score(Ytspaclitaxel_brca.astype(int), PredTpaclitaxel_brca.detach().numpy())
#print(f"TCGA - Test AUCPR (Paclitaxel, BRCA): {round(AUCTpaclitaxel_brca_aucpr,2)}")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTpaclitaxel_finetuned = dict()
AUCTpaclitaxel_brca_finetuned = dict()
AUCTpaclitaxel_aucpr_finetuned = dict()
AUCTpaclitaxel_brca_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(models_dir + f"Finetuned_Models/CNA_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETpaclitaxel = AutoencoderE_finetuned(TX_testEpaclitaxel)
   ZCTpaclitaxel = AutoencoderC_finetuned(TX_testCpaclitaxel)
   ZTTpaclitaxel = torch.cat((ZETpaclitaxel, ZCTpaclitaxel), 1)
   ZTTpaclitaxel = F.normalize(ZTTpaclitaxel, p=2, dim=0)
   PredTpaclitaxel = Clas_finetuned(ZTTpaclitaxel)
   AUCTpaclitaxel_finetuned[iter] = roc_auc_score(Ytspaclitaxel.astype(int), PredTpaclitaxel.detach().numpy())
   AUCTpaclitaxel_aucpr_finetuned[iter] = average_precision_score(Ytspaclitaxel.astype(int), PredTpaclitaxel.detach().numpy())

   ZETpaclitaxel_brca = AutoencoderE_finetuned(TX_testEpaclitaxel_brca)
   ZCTpaclitaxel_brca = AutoencoderC_finetuned(TX_testCpaclitaxel_brca)
   ZTTpaclitaxel_brca = torch.cat((ZETpaclitaxel_brca, ZCTpaclitaxel_brca), 1)
   ZTTpaclitaxel_brca = F.normalize(ZTTpaclitaxel_brca, p=2, dim=0)
   PredTpaclitaxel_brca = Clas_finetuned(ZTTpaclitaxel_brca)
   AUCTpaclitaxel_brca_finetuned[iter] = roc_auc_score(Ytspaclitaxel_brca.astype(int), PredTpaclitaxel_brca.detach().numpy())
   AUCTpaclitaxel_brca_aucpr_finetuned[iter] = average_precision_score(Ytspaclitaxel_brca.astype(int), PredTpaclitaxel_brca.detach().numpy())

max_key = max(AUCTpaclitaxel_aucpr_finetuned, key=AUCTpaclitaxel_aucpr_finetuned.get)
max_key_brca = max(AUCTpaclitaxel_brca_aucpr_finetuned, key=AUCTpaclitaxel_brca_aucpr_finetuned.get)

print(f"GDSC - Training AUC (MITOSISi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (MITOSISi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Paclitaxel): {round(AUCTpaclitaxel,2)}")
print(f"TCGA - Test AUCPR (Paclitaxel): {round(AUCTpaclitaxel_aucpr,2)}")
print(f"TCGA - Test AUC (Paclitaxel, BRCA): {round(AUCTpaclitaxel_brca,2)}")
print(f"TCGA - Test AUCPR (Paclitaxel, BRCA): {round(AUCTpaclitaxel_brca_aucpr,2)}\n")

print(f"GDSC - Training AUC (MITOSISi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (MITOSISi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Paclitaxel, Finetuned): {round(AUCTpaclitaxel_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Paclitaxel, Finetuned): {round(AUCTpaclitaxel_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (MITOSISi, Finetuned): {round(AUCT_finetuned[max_key_brca],2)}")
print(f"GDSC - Training AUCPR (MITOSISi, Finetuned): {round(AUCTaucpr_finetuned[max_key_brca],2)}")
print(f"TCGA - Test AUC (Paclitaxel, BRCA, Finetuned): {round(AUCTpaclitaxel_brca_finetuned[max_key_brca],2)}")
print(f"TCGA - Test AUCPR (Paclitaxel, BRCA, Finetuned): {round(AUCTpaclitaxel_brca_aucpr_finetuned[max_key_brca],2)}\n\n")

print("Models for maximum finetuning (Paclitaxel)")
print(f"Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"CNA_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"Class_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (Paclitaxel, BRCA)")
print(f"Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key_brca}.pt")
print(f"CNA_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key_brca}.pt")
print(f"Class_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_{max_key_brca}.pt")

GDSC - Training AUC (MITOSISi): 0.79
GDSC - Training AUCPR (MITOSISi): 0.32
TCGA - Test AUC (Paclitaxel): 0.64
TCGA - Test AUCPR (Paclitaxel): 0.79
TCGA - Test AUC (Paclitaxel, BRCA): 0.74
TCGA - Test AUCPR (Paclitaxel, BRCA): 0.94

GDSC - Training AUC (MITOSISi, Finetuned): 0.8
GDSC - Training AUCPR (MITOSISi, Finetuned): 0.32
TCGA - Test AUC (Paclitaxel, Finetuned): 0.63
TCGA - Test AUCPR (Paclitaxel, Finetuned): 0.8

GDSC - Training AUC (MITOSISi, Finetuned): 0.8
GDSC - Training AUCPR (MITOSISi, Finetuned): 0.32
TCGA - Test AUC (Paclitaxel, BRCA, Finetuned): 0.75
TCGA - Test AUCPR (Paclitaxel, BRCA, Finetuned): 0.94


Models for maximum finetuning (Paclitaxel)
Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_10.pt
CNA_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_10.pt
Class_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_10.pt

Models for maximum finetuning (Paclitaxel, BRCA)
Exprs_Paclitaxel_GDSC_TCGA_MITOSISi_Expression_CNA_Finetuned_7.pt
CNA_Paclitaxe

In [45]:
# create a dataframe with the prediction results 

ec = {"Data": ["Expr + CNA", "Expr + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
       "AUC (TCGA)": [round(AUCTpaclitaxel,2), round(AUCTpaclitaxel_finetuned[max_key],2)],
       "AUCPR (TCGA)":  [round(AUCTpaclitaxel_aucpr,2), round(AUCTpaclitaxel_aucpr_finetuned[max_key],2)],
       "AUC (TCGA-BRCA)":  [round(AUCTpaclitaxel_brca,2), round(AUCTpaclitaxel_brca_finetuned[max_key_brca],2)],
       "AUCPR (TCGA-BRCA)": [round(AUCTpaclitaxel_brca_aucpr,2), round(AUCTpaclitaxel_brca_aucpr_finetuned[max_key_brca],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
ec_dataframe = pd.DataFrame.from_dict(ec, orient='index').transpose()  
ec_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_CNA_MITOSISi_Paclitaxel.tsv",
                    sep = "\t",
                    index = False)

ec_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-BRCA),AUCPR (TCGA-BRCA),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + CNA,0.79,0.32,0.64,0.79,0.74,0.94,6360,984
1,Expr + CNA (Fine-tuned),0.8,0.32,0.63,0.8,0.75,0.94,6360,984


In [46]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=984, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [47]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=984, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [48]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=96, out_features=1, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Sigmoid()
  )
)

In [49]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_MITOSISi/Expression_CNA/Predictions


In [50]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_MITOSISi_Paclitaxel Predictions (Expression and CNA).txt', 'w')
file.write(f"GDSC Training (EC) AUC (TCGA_MITOSISi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EC) AUCPR (TCGA_MITOSISi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EC) AUC (Paclitaxel): {round(AUCTpaclitaxel,2)}\n")
file.write(f"TCGA Test (EC) AUCPR (Paclitaxel): {round(AUCTpaclitaxel_aucpr,2)}\n\n")

file.close()